# Fetching California Agency Data

In [3]:
import json
import pandas as pd
from pandas import DataFrame
import requests

Read the local json file with the API key.

In [9]:
with open("../setup.json", "r") as setup:
    api_key = json.load(setup)

{'fbikey': 'CtP1keecEgA7wkF9XvjHmYiZuiBz4Mbh3lSF9szL'}

Using the [API endpoints](https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi), we will pull down data beginning with the LE agencies in California.

In [12]:
# Request the agencies and turn this into a dictionary.
ca_agencies = requests.get(f"https://api.usa.gov/crime/fbi/cde/agency/byStateAbbr/CA?API_KEY={api_key['fbikey']}").json()
# Get CA Agencies as DataFrame for use later
ca_agencies_df = DataFrame.from_dict(ca_agencies)
ca_agencies_df.to_csv("../fbi_crime_data/CA/ca_agencies_2020.csv",
                      index=False)

Next let's grab some data at the agency level, which will require the ORIs. Let's begin by isolating ORIs.

In [13]:
oris = [i['ori'] for i in ca_agencies]

Pull number of employees for all agencies in CA:

In [14]:
def get_employees_by_ori(ori: str, key: str = None, endpoint: str = None,
                         verbose: bool = False) -> dict:
    if endpoint is None:
        endpoint = "https://api.usa.gov/crime/fbi/cde/pe/agency/"
    query = f"{endpoint}{ori}/byYearRange?from=2020&to=2020&API_KEY={key}"
    res = requests.get(query) 
    if res.status_code == 200:
        out = DataFrame.from_dict(res.json())
        return out
    else:
        if verbose:
            print(f"For {ori} no record found.")
        pass  

employees = [get_employees_by_ori(i, api_key['fbikey']) for i in oris]
employees = pd.concat(employees)
employees.to_csv("../ca_employees_2020.csv", index=False)   

Next, get arrests by ORI:

In [ ]:
def get_arrests_by_ori(ori: str, key: str = None, endpoint: str = None,
                         verbose: bool = False) -> dict:
    if endpoint is None:
        endpoint = "https://api.usa.gov/crime/fbi/cde/arrest/agency/"
    query = f"{endpoint}{ori}/all?from=2020&to=2020&API_KEY={key}"
    res = requests.get(query) 
    if res.status_code == 200:
        out = DataFrame.from_dict(res.json()['data'])
        out['ori'] = ori
        return out
    else:
        if verbose:
            print(f"For {ori} no record found.")
        pass  

arrests = [get_arrests_by_ori(i, api_key['fbikey']) for i in oris]
arrests = pd.concat(arrests)
arrests.to_csv("../ca_arrests_2020.csv", index=False)   